# Ajuste de Parâmetros

In [1]:
import pandas as pd
import os
from typing import List

In [2]:
%pip install nltk

  Using cached nltk-3.8.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached regex-2024.5.15-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
Using cached regex-2024.5.15-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (785 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Definição de Classes, opções de target
class_labels = { 'email':0, 'resume':1, 'scientific_publication':2 }

In [4]:
from nltk.corpus import stopwords
# NLTK (Natural Language Toolkit) 
# conjunto de palavras stopwords para o idioma inglês 
# Lista de palavras que são consideradas stopwords (palavras vazias) para o idioma inglês. 
stopwords_list = stopwords.words("english")
# stopwords_list = set(stopwords.words('english'))

In [5]:
from nltk.tokenize import word_tokenize
from string import punctuation
from nltk.stem import WordNetLemmatizer 
import re
from nltk.corpus import stopwords

# Download dos recursos necessários do NLTK
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
def lowercase_and_remove_special_characters(text: str) -> str:
    """
    Converte o texto para minúsculas e remove caracteres especiais.
    - Conversão para Minúsculas 
    - Substituição de Quebras de Linha e Tabulações 
    - Remoção de Espaços Extras 
    - Remoção de Números
    - Remoção de Pontuação 
    Args:
        text (str): O texto original a ser processado.
    Returns:
        str: O texto processado com minúsculas e sem caracteres especiais.
    """
    text = text.lower()
    text = text.replace("\n", " ").replace("\t", " ")
    text = re.sub("\s+", " ", text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    return text

In [7]:
def tokenize_text(text: str) -> List[str]:
    """
    Tokeniza o texto usando NLTK word_tokenize.
    Args:
        text (str): O texto a ser tokenizado.
    Returns:
        List[str]: Lista de tokens gerada a partir do texto.
    """
    tokens = word_tokenize(text)
    return tokens

In [8]:
def remove_punctuation_and_stopwords(tokens: List[str]) -> List[str]:
    """
    Remove pontuações e stopwords da lista de tokens.
    Args:
        tokens (List[str]): Lista de tokens a serem processados.
    Returns:
        List[str]: Lista de tokens sem pontuações e stopwords.
    """
    tokens = [token for token in tokens if token not in punctuation and token not in stopwords_list]
    return tokens

In [9]:
def lemmatize_tokens(tokens: List[str]) -> List[str]:
    """
    Realiza a lematização dos tokens usando WordNetLemmatizer do NLTK.
    Args:
        tokens (List[str]): Lista de tokens a serem lematizados.
    Returns:
        List[str]: Lista de tokens lematizados.
    """
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return lemmatized_tokens

In [10]:
def preprocess_data(text: str) -> str:
    """
    Realiza o pré-processamento completo do texto, incluindo minúsculas,
    remoção de caracteres especiais, tokenização, remoção de pontuações
    e stopwords, e lematização.
    Args:
        text (str): O texto original a ser pré-processado.
    Returns:
        str: O texto pré-processado final, pronto para uso em análises adicionais.
    """
    # Aplica as funções em sequência
    text = lowercase_and_remove_special_characters(text)
    tokens = tokenize_text(text)
    tokens = remove_punctuation_and_stopwords(tokens)
    final_text = lemmatize_tokens(tokens)
    return " ".join(final_text)

In [11]:
from PIL import Image
import pytesseract

def process_image(label_path: str, file_name: str) -> str:
    """
    - carrega uma imagem.png
    - extrai o texto da imagem via OCR
    - realiza o pre processamento do texto
    retorna o texto ja pre processado
    """
    # Carrega a imagem
    image = Image.open(os.path.join(label_path, file_name))
    # Usa Tesseract OCR para extrair o texto da imagem
    text = pytesseract.image_to_string(image)
    # Executa o pré-processamento do texto
    processed_text = preprocess_data(text)
    return processed_text


In [12]:
def process_images(path: str)-> pd.DataFrame:
    """
    retorna DF pandas, com duas colunas 'Text' e 'Label';
    'Text': representa os textos extraido via OCR da imagem
    'Label' representa a classe/categoria/tipo/label original desta imagem
    """
    # Diretório com as imagens
    image_folder = os.listdir(path)
    # Lista para armazenar os dados antes de criar o DataFrame
    data = []
    # Itera sobre cada diretório de classe (label_dir) ['email', 'resume','scientific_publication']
    for label_dir in image_folder:
        # Caminho completo para o diretório atual de documentos
        label_path = os.path.join(path, label_dir)
        # Itera sobre os arquivos (imagem.png) dentro do diretório
        for file_name in os.listdir(label_path):
            processed_text = process_image(label_path, file_name)
            # Obtém o rótulo numérico correspondente ao tipo de documento (label)
            label = class_labels[label_dir]
            # Adiciona um dicionário à lista de dados
            data.append({'Text': processed_text, 'Label': label})
    # Cria o DataFrame final a partir da lista de dicionários
    df = pd.DataFrame(data)        
    return df

In [13]:
def process_dataset(path: str, parquet_file: str)-> pd.DataFrame:
    """
    processa todo o dataset de imagens extensão.png covertando para dataframe pandas já pronto para usar no treinamento do modelo
    """
    # Verifica se o arquivo Parquet já existe
    if os.path.exists(parquet_file):
        # Carrega os dados existentes se o arquivo Parquet já existir
        df = pd.read_parquet(parquet_file)
    else:
        df = process_images(path)
        # Salva o DataFrame em arquivo Parquet para uso futuro não precisar reprocessar e ja pegar pronto
        df.to_parquet(parquet_file, index=False)
    return df

# Armazenar dados processos
parquet_file = "document_texts.parquet"
# Caminho para o diretório contendo as imagens
path = "dataset/www-kaggle-com_datasets_ritvik1909_document-classification-dataset"
# Chamada da função para processar o dataset e obter o DataFrame resultante
df = process_dataset(path, parquet_file)

# Separando os dados em treino e teste

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['Text'], df['Label'], test_size=0.2, random_state=42)

### Vetorizando os documentos

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Configurando o TfidfVectorizer
tfidf = TfidfVectorizer(ngram_range=(2,5), max_df=0.95, min_df=2 ,max_features=10000)

In [16]:
# Ajustando e transformando os dados de treinamento
tfidf_train = tfidf.fit_transform(X_train)
# Convertendo a matriz esparsa em uma matriz densa
tfidf_test = tfidf.transform(X_test)

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Lista de modelos para comparar
models = {
    'Logistic Regression': LogisticRegression(),
    'Random Forest': RandomForestClassifier(),
    'SVM': SVC(),
    'Naive Bayes': MultinomialNB(),
    'KNN': KNeighborsClassifier(),
    'Gradient Boosting': GradientBoostingClassifier()
}

# Ajuste de hiperparâmetros usando GridSearchCV

busca pela configuração ideal de parâmetros para cada modelo.

Definição de param_grids: Para cada modelo, definimos um dicionário de param_grid que contém os parâmetros que desejamos ajustar utilizando GridSearchCV. Os parâmetros específicos foram escolhidos com base na prática comum e podem ser ajustados conforme necessário.

Loop de Ajuste de Parâmetros: Iteramos sobre o dicionário models, que contém os modelos a serem testados. Para cada modelo, realizamos o ajuste de hiperparâmetros usando GridSearchCV.

GridSearchCV: Criamos um objeto GridSearchCV para cada modelo, onde especificamos o modelo, o param_grid correspondente, o número de folds para validação cruzada (cv=5 neste exemplo) e a métrica de avaliação (scoring='accuracy').

Execução da Busca em Grade: Chamamos o método fit() para executar a busca em grade no conjunto de treinamento (tfidf_train, y_train).

Armazenamento do Melhor Modelo: Após a busca em grade, armazenamos o melhor modelo encontrado (best_estimator_) no dicionário best_models.

Avaliação no Conjunto de Teste: Finalmente, avaliamos os melhores modelos encontrados no conjunto de teste (tfidf_test, y_test) e exibimos a acurácia para cada modelo.

In [21]:
param_grid_lr = {
    'C': [0.1, 1.0, 10.0],
    'penalty': ['l2'],
    'solver': ['liblinear']
}

param_grid_rf = {
    'n_estimators': [100],
    'max_depth': [None],
    'min_samples_split': [2]
}

param_grid_svc = {
    'C': [1.0],
    'kernel': ['linear'],
    'gamma': ['scale']
}

param_grid_nb = {
    'alpha': [0.1],
    'fit_prior': [True]
}

param_grid_knn = {
    'n_neighbors': [10],
    'weights': ['uniform'],
    'metric': ['euclidean']
}

param_grid_gb = {
    'n_estimators': [200],
    'learning_rate': [0.5],
    'max_depth': [10]
}

In [20]:
from sklearn.model_selection import GridSearchCV

# Definir os parâmetros que você deseja testar para cada modelo
param_grid_lr = {
    'C': [0.1, 1.0, 10.0],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga']
}

param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

param_grid_svc = {
    'C': [0.1, 1.0, 10.0],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto']
}

param_grid_nb = {
    'alpha': [0.1, 0.5, 1.0],
    'fit_prior': [True, False]
}

param_grid_knn = {
    'n_neighbors': [3, 5, 10],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

param_grid_gb = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.05, 0.1, 0.5],
    'max_depth': [3, 5, 10]
}

# Dicionário de param_grids para cada modelo
param_grids = {
    'Logistic Regression': param_grid_lr,
    'Random Forest': param_grid_rf,
    'SVM': param_grid_svc,
    'Naive Bayes': param_grid_nb,
    'KNN': param_grid_knn,
    'Gradient Boosting': param_grid_gb
}

# Lista para armazenar os melhores modelos ajustados
best_models = {}

# Executar GridSearchCV para cada modelo
for name, model in models.items():
    print(f"Ajustando hiperparâmetros para {name}...")
    
    # Definir o param_grid específico para o modelo atual
    param_grid = param_grids[name]
    
    # Criar o objeto GridSearchCV
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
    
    # Executar a busca em grade no conjunto de treinamento
    grid_search.fit(tfidf_train, y_train)
    
    # Armazenar o melhor modelo ajustado
    best_models[name] = grid_search.best_estimator_
    
    # Mostrar os melhores parâmetros encontrados
    print(f"Melhores parâmetros encontrados para {name}:")
    print(grid_search.best_params_)
    print()

# Avaliar os melhores modelos no conjunto de teste e mostrar métricas
print("\nAvaliação dos melhores modelos no conjunto de teste:")

for name, model in best_models.items():
    # Avaliar o modelo no conjunto de teste
    accuracy = model.score(tfidf_test, y_test)
    print(f'{name}: Acurácia no teste = {accuracy:.4f}')


Ajustando hiperparâmetros para Logistic Regression...


/opt/conda/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which 

Melhores parâmetros encontrados para Logistic Regression:
{'C': 1.0, 'penalty': 'l2', 'solver': 'liblinear'}

Ajustando hiperparâmetros para Random Forest...
Melhores parâmetros encontrados para Random Forest:
{'max_depth': None, 'min_samples_split': 2, 'n_estimators': 100}

Ajustando hiperparâmetros para SVM...
Melhores parâmetros encontrados para SVM:
{'C': 1.0, 'gamma': 'scale', 'kernel': 'linear'}

Ajustando hiperparâmetros para Naive Bayes...
Melhores parâmetros encontrados para Naive Bayes:
{'alpha': 0.1, 'fit_prior': True}

Ajustando hiperparâmetros para KNN...
Melhores parâmetros encontrados para KNN:
{'metric': 'euclidean', 'n_neighbors': 10, 'weights': 'uniform'}

Ajustando hiperparâmetros para Gradient Boosting...
Melhores parâmetros encontrados para Gradient Boosting:
{'learning_rate': 0.5, 'max_depth': 10, 'n_estimators': 200}


Avaliação dos melhores modelos no conjunto de teste:
Logistic Regression: Acurácia no teste = 0.8182
Random Forest: Acurácia no teste = 0.9091
SVM